# HARDI Probabilistic Tractography (exercise)

In this tutorial, let's implement an procedure to perform propabilistic tractography from ODFs estimated using the Q-ball Constant Solid Angle (CSA) model. For this complete below the empty notebook cells regarding the following steps: 

1) Sample directions for the CSA-ODF estimates using the ProbabilisticDirectionGetter.

2) Use the same seeds of our previous tutorials to begin propagating streamlines in the corpus callosum.

3) The stopping criteria for streamline propagation is defined based on the General fractional anisotropy of q-ball model.

4) We use the same method for propagating streamlines.

Loading the Stanford's dMRI dataset and its anotated labels.

In [1]:
import numpy as np

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, get_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data

hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

label_fname = get_fnames('stanford_labels')
labels = load_nifti_data(label_fname)
white_matter = (labels == 1) | (labels == 2)

As for the previous examples, we mask our loaded data:

In [2]:
from dipy.segment.mask import median_otsu

maskdata, mask = median_otsu(data, vol_idx=range(0, 9),
                             numpass=1, dilate=5)

### 1) Getting directions from the dMRI dataset based on CSA model

Remember for this you need to import the CsaOdfModel from dipy.reconst.shm and the ProbabilisticDirectionGetter from dipy.direction

In [3]:
from dipy.reconst.shm import CsaOdfModel
from dipy.direction import ProbabilisticDirectionGetter
from dipy.data import get_sphere

sphere = get_sphere('repulsion724')

csa_model = CsaOdfModel(gtab, sh_order=6)
csa_fit = csa_model.fit(data, mask=white_matter)

odf = csa_fit.odf(sphere)
pmf = odf.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30.,
                                                sphere=sphere)

### 2) Defining the seeds

As for the other examples,place a grid of 2 × 2 × 2 grid of seeds per voxel, in a sagittal slice of the corpus callosum. 

In [4]:
from dipy.tracking import utils

seed_mask = (labels == 2)
seeds = utils.seeds_from_mask(seed_mask, affine, density=[2, 2, 2])

### 3) Defining the tracking stopping criterion

Use the general fractional anisotropy estimates of the CSA q-ball model to define the stopping criterion.

In [5]:
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

stopping_criterion = ThresholdStoppingCriterion(csa_fit.gfa, .2)

### 4) Propagating the streamlines

Having the directions, starting position, and stopping criterion defined we can start generating streamlines.

In [6]:
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines

streamlines_generator = LocalTracking(prob_dg, stopping_criterion, seeds,
                                      affine=affine, step_size=.5)
streamlines = Streamlines(streamlines_generator)

Plot the resulting streamlines...

In [7]:
from dipy.viz import window, actor, has_fury, colormap

if has_fury:
    # Prepare the display objects.
    color = colormap.line_colors(streamlines)

    streamlines_actor = actor.line(streamlines,
                                   colormap.line_colors(streamlines))

    # Create the 3D display.
    scene = window.Scene()
    scene.add(streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    window.record(scene, out_path='csa_probabilistic_tractogram.png', size=(800, 800))
    window.show(scene)

C:\Users\rafae\anaconda3\envs\dipy_release\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


Saving the tractogram...

In [8]:
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk

sft = StatefulTractogram(streamlines, hardi_img, Space.RASMM)
save_trk(sft, "csa_CC_tractography_probabilistic.trk", streamlines)